# Graphene / metal oxide powders x-ray diffraction

## Comparison of x-ray diffractions in the process of heating the samples

### Plot componds of your choice
Select which compound you want to plot and to be displayed in normal or logarithmic scale

You can specify whether to show the plots in jupyter or save the on your computer

In [ ]:
import itertools
import notebook_finder
import xray_plots_helper_classes as xphc


# ****** PLOT OPTIONS ******

# here your can set how individual plots will show in figure in relation to each other
# plotHeight, plotDistance, offset parameters in percentage [%] of figure height
    # plotHeight ~> height of each plot in relation to the figure height
    # plotDistance ~> how far are the plots spaced between each other f.ex. between 550ºC and  650ºC
    # offset ~> the level at which the first plot will show above x axis
plotsOptions = xphc.PlotsOptions(30, 24, 8)

# files and target paths
plotsOptions.xrdFilesPath = 'xrd/'
plotsOptions.bgImagesPath = 'SEM/'
plotsOptions.saveFiguresDestinationPath = 'figures/before and after heating/'

plotsOptions.bgImagesFileNameSufixAndExtension = '_SEM_AfterElectrolysis.tif'
plotsOptions.savedFiguresFileExtension = '.png'

plotsOptions.legendBoxOpacity = 0.5
plotsOptions.bgImageOpacity = 0.2


# ****** PLOT COLORS ******

# color themes ~> to modify a color theme, copy existing one and insert your changes there
#                                      CuO/CuCl2  ZnO/CuCl2  Zno/FeCl3                   CuO/CuCl2  ZnO/CuCl2  Zno/FeCl3
themes_first_suggestions = {'theme' : ["#960018", "#456456", "#FFE5B4"], 'background' : ["#faeeee", "#defdef", "#4B201A"]}
themes_from_xrayholders  = {'theme' : ["#faeeee", "#defdef", "#FFE5B4"], 'background' : ["#6D3721", "#CCB78A", "#4B201A"]}

# colors of the plots
plotsColors_ver1 = ['#24fafa', '#fa2436', '#124872']

# select your color theme from the options above
theme = themes_from_xrayholders
plotsColors = plotsColors_ver1


# ****** SELECTION OF POWDERS ******

# powders to select from 
electroCuOCucl2 = xphc.PowderAfterElectrolysis('CuO', 'CuCl2', 'Cu', theme['background'][0], theme['theme'][0])
electroZnOCucl2 = xphc.PowderAfterElectrolysis('ZnO', 'CuCl2', 'Cu', theme['background'][1], theme['theme'][1])
electroZnOFeCl3 = xphc.PowderAfterElectrolysis('ZnO', 'FeCl3', 'Steel', theme['background'][2], theme['theme'][2])

# select compounds you would like to have plotted 
chosenFormulas = [electroZnOCucl2, electroZnOFeCl3, electroCuOCucl2]


# ****** LOG AND/OR NORMAL PLOTS & SHOW OR SAVE FIGURES ******

# Choose to show Normal y axis [False], log y axis [true] or both [False, True]
isLogarithmic = [False, True]

# Choose whether to save figures on the disk [True] or show in notebook [False]
saveOnDisk = False


# ****** PLOT RESULTS ******

# iterate through combinations and make figuresx
for x in list(itertools.product(chosenFormulas, isLogarithmic)):
    xphc.plotBeforeAfterHeating(x[0], x[1], saveOnDisk, plotsColors, plotsOptions)
    
    